## Parse w32_mal_report and hooklogs
* w32_mal_report is from Virustotal (total 357 malware/binaries)
* hooklohgs are from dynamic analysis (total 420 files/processes)

In [1]:
import os
import json

w32_mal_report = 'data/w32_mal_report/'
tag = 'zusy_'
hooklogs = 'data/'+tag+'hooklogs/'

# MIKE: if text_mining is True, the result strings will be stripped.
text_mining = True
csv_file = 'output/w32_mal_report_with_hooklog_tm_'+tag+'.csv' if text_mining else 'output/w32_mal_report_with_hooklog_'+tag+'.csv'

In [2]:
import string
def _isHex(s):
    if len(s) == 0:
        return False
    for c in s:
        if c not in string.hexdigits:
            return False
    return True

### w32_mal_report
First run, get malware list

In [3]:
malware_file_list = next(os.walk(w32_mal_report))[2]
mal_in_rep_list = list()
for r in malware_file_list:
    mal_in_rep_list.append(r[:-len('.txt')])

print 'mal_in_rep_list', len(mal_in_rep_list) # number of malware in the w32_mal_report

#=======

# But not all the malware in the report is Win32, so ...
for f in malware_file_list:
# for macOSX file system    
    print type(f)
    print f
    if f == ".DS_Store":
        print "find '.DS_Store file', instruction - continue."
        mal_in_rep_list.remove('.DS_S')
        continue
    
    h = open(w32_mal_report + f, 'r')
    json_report = h.read()
    if h: h.close()
    malware = f[:-len('.txt')]
    
    d = json.loads(json_report)
    if(d['type'] != 'Win32 EXE'):
        print malware, 'is not win32. Remove from mal_in_rep_list.'
        mal_in_rep_list.remove(malware)

print 'mal_in_rep_list', len(mal_in_rep_list) # number of malware in the w32_mal_report

mal_in_rep_list 357
<type 'str'>
0006dfb19193ff5910b3ffc94a29a7f6.txt
<type 'str'>
005528c99846dfd84e7a29ae2cf18fd8.txt
<type 'str'>
010ac3a6f8c7dba927a2ca5d91fe224e.txt
010ac3a6f8c7dba927a2ca5d91fe224e is not win32. Remove from mal_in_rep_list.
<type 'str'>
0172fef366185da8add30d714f346a35.txt
<type 'str'>
0278192ce1aa7f8c4ae746eae73e6475.txt
<type 'str'>
02c27a2e4c215d22fabdfc7b46fd6f20.txt
<type 'str'>
0311b7b8b878d337b7cf829af7548093.txt
<type 'str'>
03c009b451e012002513513a5d7e1b00.txt
<type 'str'>
043121379d4a5f0f227d74ef0336428c.txt
<type 'str'>
060c18ffe4b4f8b47c90a2339533d622.txt
<type 'str'>
06dc60ab51a3ba87757726f7a085d9d5.txt
<type 'str'>
08261732bfcae9a06c7fd6e6f76b2a81.txt
<type 'str'>
1000511f9f867a3ee51cb91e7d922e99.txt
<type 'str'>
1036837a1c94b6568a6596077452a19a.txt
<type 'str'>
1037b8385f3eb7f7ed6cbdf4251b5fa2.txt
<type 'str'>
10438da7c2166156c4fbce788e4793ed.txt
<type 'str'>
1088046dcb34429d55d4695cf3b28301.txt
<type 'str'>
109384695f396badeedd8e171b31b734.txt
<typ

### hooklogs
First run, get process list and the malware list. Note that the some hooklog has shorten name.

In [4]:
process_file_list = next(os.walk(hooklogs))[2]
proc_in_log_list = list()
mal_in_log_set = set()
for p in process_file_list:
    m_p = p[:-len('.trace.hooklog')]
    proc_in_log_list.append(m_p)
    mal_in_log_set.add(m_p.split('_')[0])

print 'mal_in_log_set', len(mal_in_log_set) # number of malware in the hooklog
print 'proc_in_log_list', len(proc_in_log_list) # number of processes in the hooklog

mal_in_log_set 4
proc_in_log_list 8


### Intersection
I have no idea which list is correct, so make an intersection to make sure the malware/process list is correct.

In [5]:
#MIKE: if hooklog's name is not shorten, the line below is good enough. However, ...
#Find correct malware, than correct process
#inter_list = set(mal_in_rep_list).intersection(mal_in_log_set)
#print len(inter_list)

# Make a dict, d[sort name] = long_name; Make a true process list
proc_list = proc_in_log_list
m_set = mal_in_log_set
mal_dict = dict()

#os.path.getsize(hooklogs + process + '.trace.hooklog') < 2000:

for m in list(m_set): # could be short name
    _done = False
    for n in mal_in_rep_list: # long name    #==============not in rep 不用for
        if m == n:
            mal_dict[m] = n
            _done = True
            break
        elif m in n:
            mal_dict[m] = n
            _done = True
            break
    if not _done: # cannot find corresponding item
        print 'no such', m, 'in report.'
        m_set.remove(m)

for m_p in proc_in_log_list:
    m = m_p.split('_')[0]
    if m not in mal_dict.keys():
        proc_list.remove(m_p)

print 'mal_dict', len(mal_dict) # MIKE: the malware we want to deal with
print 'proc_list', len(proc_list) # MIKE: the corresponding processes

mal_dict 4
proc_list 8


In [6]:
#=======
# 1) check again, using long_name
# 2) do text_mining ds at the same time

import re
tag_dict = dict()
delimiter = '\,|!|\(|\)|\[|\]|@|:|/|\.| '

av_set = set()
for malware in mal_dict.values():
    h = open(w32_mal_report + malware + '.txt', 'r')
    json_report = h.read()
    if h: h.close()
    
    d = json.loads(json_report)
    for c, s in enumerate(d['scans'].keys()):
        scan_result = d['scans'].get(s)
        if scan_result.get("detected") == True:
            engine = s.encode('ascii', 'ignore')
            result = scan_result.get("result").encode('ascii', 'ignore')
            result = result.replace(',', '') # special replacement for csv
            av_set.add(engine)
            
            if text_mining:
                tokens = re.split(delimiter, result.lower())
                for k in tokens:
                    if len(k) == 0:
                        continue
                    if k in tag_dict:
                        tag_dict[k] += 1
                    else:
                        tag_dict[k] = 1

av_list = list(av_set)
print 'av_list', len(av_list), av_list

av_list 48 ['Bkav', 'MicroWorld-eScan', 'nProtect', 'CAT-QuickHeal', 'McAfee', 'Malwarebytes', 'K7AntiVirus', 'K7GW', 'TheHacker', 'Agnitum', 'F-Prot', 'Norman', 'ESET-NOD32', 'Avast', 'ClamAV', 'Kaspersky', 'BitDefender', 'NANO-Antivirus', 'SUPERAntiSpyware', 'ByteHero', 'Ad-Aware', 'Emsisoft', 'Comodo', 'F-Secure', 'DrWeb', 'VIPRE', 'AntiVir', 'McAfee-GW-Edition', 'Sophos', 'Cyren', 'Jiangmin', 'Avira', 'Antiy-AVL', 'Kingsoft', 'Microsoft', 'ViRobot', 'GData', 'Commtouch', 'AhnLab-V3', 'VBA32', 'AVware', 'Panda', 'Rising', 'Ikarus', 'Fortinet', 'AVG', 'Baidu-International', 'Qihoo-360']


In [7]:
#Special handel for tag_dict
general_string = ['win32','trojan','adware','generic','application','variant','downloader','not-a-virus','downware',
                 'unwanted-program','heur','troj','bundler','antifw','riskware','optional','malware','behaveslike',
                 'kcloud','agent','trojandownloader','appl','trojware','installer','trojan-downloader','virus',
                 'backdoor','injector','malware-cryptor','dropper','cryptor','bundleapp','suspicious','antifwk',
                 'adinstaller','crypt','bundleinstaller','xpack', 'hacktool','patcher','troj_gen','grayware',
                 'software','install','click','heuristic','packed','unknown','applicunwnt','dropped','trojan-clicker',
                 'net-worm','monitoringtool','worm','tool','toolbar','eldorado','autorun','hw32', 'trojan-dropper']
special_string = ['kdz', 'ipz', 'lmn']
tag_key_list =  tag_dict.keys()

In [8]:
# Remove tags that have only 1 count, or ...

for k in tag_key_list:
    if  k in special_string:
        continue
    elif _isHex(k):
        del tag_dict[k]
    elif tag_dict[k] <= 1 or len(k) <= 3:
        del tag_dict[k]
    elif k in general_string:
        del tag_dict[k]
     
print 'tag_dict', len(tag_dict), tag_dict.keys()

tag_dict 21 ['loadmoney', 'ogimant', 'undef', 'crypt2', 'malex', 'sisproc', 'pup-fnb', 'newmalware-rootkit-i-based', 'startpage', 'swisyn', 'lmn', 'inject1', 'kryptik', 'qvm20', 'badur', 'ldmon-d', 'zusy', 'servstart', 'limpopo', 'cdic', 'dchp']


### Make csv and pickle files

In [9]:
o = open(csv_file, 'w')
# header
o.write("%s,%s,%s,%s,%s" % ('Hash', 'Type', 'First_Seen', 'Scan', 'Tag'))
map(lambda s: o.write(',%s' % s), av_list)
o.write('\n')
import pickle
malwareFami_dict={}    #==========

#each malware
for malware in mal_dict.values():
    h = open(w32_mal_report + malware + '.txt', 'r')
    json_report = h.read()
    if h: h.close()
    
    d = json.loads(json_report)
    # meta for each malware
    o.write("%s,%s,%s,%d/%d" % (malware, d['type'], d['first_seen'], d['positives'], len(d['scans'])))
    
    # meta Tag, no ','
    tags = ''
    this_tag_set = set()
    
    # arrange result
    candidate_dict = dict()
    for c, s in enumerate(d['scans'].keys()):
        scan_result = d['scans'].get(s)
        if scan_result.get("detected") == True:
            engine = s.encode('ascii', 'ignore')
            result = scan_result.get("result").encode('ascii', 'ignore')
            candidate_dict[engine] = result
            
    # output av result for this malware
    out = ''
    for av in av_list:
        if av in candidate_dict:
            result = candidate_dict[av].replace(',', '') # special replacement for csv
            if text_mining:
                result_tag = ''
                tokens = re.split(delimiter, result.lower())
                for k in tokens:
                    if k in tag_dict:
                        if k not in this_tag_set:
                            this_tag_set.add(k)
                            tags += (k+';')
                        result_tag += (k+'.')
                if result_tag:
                    result_tag = result_tag[:-1] # remove last .
                out += (',' + result_tag)
            else:
                out += (',' + result)
        else:
            out += ','
    if tags:
        tags = tags[:-1] # remove last ;
    
    print malware, tags.split(';')[0]  #==========
    malwareFami_dict[malware] = tags.split(';')[0]  #==========
    o.write(',' + tags + out + '\n')

if o: o.close()

with open('pickle/av_list-mal_dict-family_dist_'+tag+'.pickle', 'wb') as o:
    pickle.dump(malwareFami_dict, o)
if o: o.close()

82cf1e67ed155da7e89fc4f49b97050d zusy
833562ee3e3a2569fe42a303d5619464 zusy
808ab4f1cca8091b05d26f15f775c525 zusy
c3ea3ad31e662953f0fcd1c62c50b130 zusy


In [10]:
import pickle

data = [av_list, mal_dict, proc_list]

with open('pickle/av_list-mal_dict-proc_list_'+tag+'.pickle', 'wb') as o:
    pickle.dump(data, o)